# Siamese Network Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WillzWayn/growing-happy/AI/AI/Siameses Tower/ContrastiveLoss - Pytorch - Siameses Tower.ipynb)



googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb

In this notebook i will walk through the creation of a Siamese Neural Network, which is often used for tasks like image similarity (signature verification, face matching, ...), text similarity, and one-shot learning.

In this case, we will use the MNIST dataset to build the network, which will compare pairs of images and determine whether they belong to the same class or not.

## Step 1: Import Required Libraries
We will start by importing necessary libraries for data manipulation, neural networks, and visualization.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torch import optim

from tqdm import tqdm

## Setup Training

In [2]:
EPOCHS = 15
EMBEDDING_SPACE = 3

## Step 2: Set Up Device
Here, we define whether we will be using a GPU (CUDA) or CPU for our computations. If a GPU is available, it will be used.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## Step 3: Create a Custom Dataset for Siamese Networks
We will now create a custom dataset class `SiameseDataset`. This dataset will provide pairs of images from the MNIST dataset, where each pair will either belong to the same class or to different classes. The label will indicate whether the images are similar (0 for same class, 1 for different classes).

In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, data, transform=None):
        """
        Initializes the SiameseDataset class.
        
        Args:
            data: List of tuples where each tuple contains (image, label).
            transform: Optional transform to be applied on a sample.
        """
        self.data = data  # Store the dataset.
        self.transform = transform  # Store the image transformation function.

    def _get_image_from_same_class(self, labelA):
        labelB = -1
        while labelB != labelA:
            imgB, labelB = random.choice(self.data)
        return imgB, labelB
    
    def _get_image_from_different_class(self, labelA):
        labelB = labelA
        while labelB == labelA:
            imgB, labelB = random.choice(self.data)
        return imgB, labelB
    
    def __getitem__(self, index):
        """
        Returns a sample from the dataset, consisting of two images and a label indicating
        if they belong to the same class.
        
        Args:
            index: Index of the first image in the dataset.
        
        Returns:
            imgA, imgB: A pair of images.
            label: A binary tensor indicating whether the images are from the same class (0) 
                   or different classes (1).
        """
        imgA, labelA = self.data[index]
        same_class_flag = random.randint(0, 1)

        if same_class_flag:
            imgB, labelB = self._get_image_from_same_class(labelA)
        else:
            imgB, labelB = self._get_image_from_different_class(labelA)

        if self.transform:
            imgA = self.transform(imgA)
            imgB = self.transform(imgB)

        return imgA, imgB, torch.tensor([labelA != labelB], dtype=torch.float32)

    def __len__(self):
        return len(self.data)
    

mnist_train = MNIST(root='./data', train=True, download=True)
mnist_test = MNIST(root='./data', train=False, download=True)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=15)
])

siamese_train = SiameseDataset(mnist_train, transform)
siamese_test = SiameseDataset(mnist_test, transform)

## Step 4: Visualize the Dataset
Let's visualize a few samples of image pairs from our custom `SiameseDataset`. This will help us understand how the pairs are generated (same class or different class).

In [4]:
def plot_siamese_samples(dataset, num_samples=5):
    """
    Plots pairs of images from the SiameseDataset along with their label indicating
    if they belong to the same class or different classes.
    
    Args:
        dataset: An instance of the SiameseDataset.
        num_samples: Number of image pairs to plot.
    """
    fig, axes = plt.subplots(num_samples, 2, figsize=(8, num_samples * 2))
    fig.suptitle('Siamese Dataset Image Pairs')

    for i in range(num_samples):
        imgA, imgB, label = dataset[random.randint(0, len(dataset)-1)]
        imgA = imgA.cpu().squeeze().numpy()
        imgB = imgB.cpu().squeeze().numpy()

        axes[i, 0].imshow(imgA, cmap='gray')
        axes[i, 0].axis('off')
        axes[i, 0].set_title(f'Image A - LABEL: {label}')

        axes[i, 1].imshow(imgB, cmap='gray')
        axes[i, 1].axis('off')
        axes[i, 1].set_title('Image B')

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.savefig('plot_siamese_samples.png')

# Example usage:
plot_siamese_samples(siamese_train, num_samples=5)

NameError: name 'siamese_train' is not defined

## Step 5: Build the Siamese Network Architecture
The `SiameseNetwork` consists of two branches that share weights. These branches process the two input images, and the results are compared to determine the similarity. The architecture consists of convolutional layers followed by fully connected layers.
import torch.nn as nn

In [5]:
class Tower(nn.Module):
    """
    This is the embedding generator.
    """
    def __init__(self):
        super(Tower, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
        )

        self.fc = nn.Sequential(
            nn.Linear(256 * 3 * 3, 1024),
            nn.ReLU(inplace=True),

            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),

            nn.Linear(256, EMBEDDING_SPACE)
        )

    def forward(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.tower = Tower()  # Shared tower network

    def forward(self, inputA, inputB):
        outputA = self.tower(inputA)
        outputB = self.tower(inputB)
        return outputA, outputB

## Step 6: Define the Contrastive Loss Function
We use the contrastive loss function to optimize the Siamese Network. This loss function computes the distance between two outputs and penalizes the network based on whether they are from the same class or not.

- **The margin forces samples from different classes to be at least a certain distance apart**; if the euclidean_distance < margin, the difference is positive, penalizing the model, with the penalty increasing as the distance decreases.
- **A lower loss** indicates that embeddings of the same class are close together, and embeddings of different classes are appropriately separated.
- **A higher loss** would indicate that embeddings of the same class are far apart or that embeddings of different classes are too close.

In [8]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=3.0):
        # The margin is the threshold
        # that determines how far apart the embeddings
        # of different classes should be.
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, outputA, outputB, label):
        # The forward method defines the computation of the contrastive loss.
        # It receives two outputs (outputA, outputB) and a label (1 or 0).
        # 'outputA' and 'outputB' represent the embeddings for two input samples.

        # Compute the Euclidean distance between the two outputs (embeddings).
        euclidean_distance = F.pairwise_distance(outputA, outputB, keepdim=True)

        # Loss for pairs that belong to the same class (label = 0). This encourages embeddings
        # from the same class to be close to each other by minimizing the squared distance.
        same_class_loss = (1 - label) * (euclidean_distance ** 2)

        # Loss for pairs that belong to different classes (label = 1). This encourages embeddings
        # from different classes to be separated by at least 'margin' by penalizing distances 
        # below the margin. The clamp ensures the difference doesn't go below 0.
        diff_class_loss = (label) * (torch.clamp(self.margin - euclidean_distance, min=0.0) ** 2)

        # Return the average loss across the batch.
        return torch.mean(same_class_loss + diff_class_loss)

In [9]:
# Example of input in mock data
# Two embeddings of size (batch_size, embedding_dimension)
outputA = torch.tensor([[0.5, 1.0], [0.8, 0.6], [1.5, 1.3]])  # Embeddings for the first set of samples
outputB = torch.tensor([[0.4, 0.9], [0.2, 0.1], [1.4, 1.2]])  # Embeddings for the second set of samples

# Labels indicating if they belong to the same class or not
# 0 means same class, 1 means different class
label = torch.tensor([[0], [1], [0]])  # Batch size = 3

# Calculate the loss
criterion = ContrastiveLoss()
loss = criterion(outputA, outputB, label)
print(loss)

tensor(1.6546)


## Step 7: Training the Network
Now we will train the network using the `SiameseDataset` and the `ContrastiveLoss`. The training loop involves computing the loss for each image pair and updating the model parameters.

In [ ]:
train_dataloader = DataLoader(siamese_train, shuffle=True, batch_size=64)

model = SiameseNetwork().to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    total_loss = 0
    
    for imgA, imgB, label in tqdm(train_dataloader, desc=f"Train epoch: {epoch+1}"):
        imgA, imgB, label = imgA.to(device), imgB.to(device), label.to(device)
        optimizer.zero_grad()
        outputA, outputB = model(imgA, imgB)
        loss_contrastive = criterion(outputA, outputB, label)
        loss_contrastive.backward()

        total_loss += loss_contrastive.item()
        optimizer.step()

    print(f"Epoch {epoch}; Loss {total_loss}")

def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for imgA, imgB, label in dataloader:
            imgA, imgB, label = imgA.to(device), imgB.to(device), label.to(device)
            outputA, outputB = model(imgA, imgB)
            euclidean_distance = F.pairwise_distance(outputA, outputB)
            predictions = euclidean_distance > 1.0  # Threshold for same/different class
            total_correct += (predictions == label).sum().item()
            total_samples += label.size(0)

    accuracy = total_correct / total_samples
    return accuracy

test_dataloader = DataLoader(siamese_test, shuffle=False, batch_size=64)
accuracy = evaluate_model(model, test_dataloader)
print(f'Test Dataset Accuracy: {accuracy}%')

def plot_siamese_predict_samples(dataset, num_samples=5):
    """
    Plots pairs of images from the SiameseDataset along with their label indicating
    if they belong to the same class or different classes.
    
    Args:
        dataset: An instance of the SiameseDataset.
        num_samples: Number of image pairs to plot.
    """
    # Set up the plot with 2 columns for image pairs
    fig, axes = plt.subplots(num_samples, 2, figsize=(8, num_samples * 2))
    fig.suptitle('Siamese Dataset Image Pairs')

    for i in range(num_samples):
        # Randomly pick an index from the dataset
        imgA, imgB, label = dataset[random.randint(0, len(dataset)-1)]
        imgA, imgB, label = imgA.to(device), imgB.to(device), label.to(device)
        outputA, outputB = model(imgA.unsqueeze(0), imgB.unsqueeze(0))
        similarity_score = F.pairwise_distance(outputA, outputB).cpu().item()

        # Convert tensors to numpy arrays for plotting
        # Move tensors to CPU before converting to NumPy arrays
        imgA = imgA.cpu().squeeze().numpy()  # Changed line
        imgB = imgB.cpu().squeeze().numpy()  # Changed line

        # Display first image (imgA)
        axes[i, 0].imshow(imgA, cmap='gray')
        axes[i, 0].axis('off')
        axes[i, 0].set_title(f'SCORE: {similarity_score}')
        PRED_LABEL = 'Same Label' if similarity_score < .35 else 'DIFF Label'
        # Display second image (imgB)
        axes[i, 1].imshow(imgB, cmap='gray')
        axes[i, 1].axis('off')
        axes[i, 1].set_title(f'PREDICT: {PRED_LABEL}')

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.savefig('plot_siamese_predict_samples.png')


plot_siamese_predict_samples(siamese_test, num_samples=5)



## Save the Model

In [ ]:
# Salvar o modelo da Tower
torch.save(model.tower, 'tower_model_siamese.pth')

## Step 8: Load the Network
Let's test the network on a few image pairs from the test dataset and visualize the similarity scores.

```python
import torch.nn as nn
from torchvision.datasets import MNIST
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
```

In [ ]:
tower_model = torch.load('tower_model_siamese.pth', map_location=device)
tower_model.eval()  # Eval mode

## Step 8: Test the Network
Let's test the network on a few image pairs from the test dataset and visualize the similarity scores.

In [ ]:
mnist_test = MNIST(root='./data', train=False, download=True)
test_tensor = torch.utils.data.TensorDataset(torch.tensor(mnist_test.data), torch.tensor(mnist_test.targets))
# Create a data loader from the tensor dataset
test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=1, shuffle=False)


# EVAL USANDO TOWER:
for imgA, label in test_tensor:
    imgA = imgA.type(torch.float32).unsqueeze(0).unsqueeze(0).to(device)  
    outputA = tower_model(imgA)
    print(outputA)
    break

for imgA, label in test_loader:
    imgA = imgA.type(torch.float32).unsqueeze(0).to(device)  
    outputA = tower_model(imgA)
    print(outputA)
    break

In [ ]:

## Visualize the embeddings.
from sklearn.manifold import TSNE

# Define a function to extract embeddings from the tower model
# Create a data loader from the tensor dataset
# Define a function to extract embeddings from the tower model
def extract_embeddings(loader, model, device):
    embeddings = []
    labels = []
    with torch.no_grad():
        for x, y in loader:
            x = x.type(torch.float32)
            x = x.to(device)
            output = model(x.unsqueeze(0))
            embeddings.append(output.cpu().numpy())
            labels.append(y.cpu().numpy())
    return np.concatenate(embeddings), np.concatenate(labels)

embeddings, labels = extract_embeddings(test_loader, tower_model, device)

In [ ]:
# Use t-SNE to reduce the dimensionality of the embeddings
def visualizate_embeddings(embeddings, labels, annotation_sample=.06):
    # Use t-SNE to reduce the dimensionality of the embeddings
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings)

    # Plot the embedding space
    plt.figure(figsize=(8, 8))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels)

    # Sample x% of the labels for annotation
    sample_idx = np.random.choice(range(len(labels)), size=int(annotation_sample * len(labels)), replace=False)
    for i in sample_idx:
        x, y = embeddings_2d[i, 0], embeddings_2d[i, 1]
        plt.annotate(str(labels[i]), (x, y), textcoords="offset points", xytext=(0,10), ha='center')

    plt.title('MNIST Embedding Space')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.colorbar()
    plt.savefig('ContrastiveLoss - MNIST Embedding Space.png')

visualizate_embeddings(embeddings, labels)